# 第8单元: 使用PyTorch编写Proximal Policy Optimization(PPO) 🤖️

在本单元中, 你将学习**使用PyTorch从头开始编写你的PPO智能体.**

为了测试鲁棒性, 我们将在2个不同的经典环境进行训练:

* [CartPole-v1](https://www.gymlibrary.ml/environments/classic_control/cart_pole/?highlight=cartpole)
* [LunarLander-v2 🚀](https://www.gymlibrary.ml/environments/box2d/lunar_lander/)

我们通过深入了解PPO的工作原理来完成课程的基础部分. 在第1单元, 你学习了在LunarLander-v2上训练PPO智能体. 但是现在, 第8单元, 你可以从头开始编写代码. 这真是太不可思议了 🤩.

![cover.jpg](./assets/cover.jpg)

⬇️ 这是你将在几分钟内实现的目标的示例([原始视频1下载链接](https://huggingface.co/sb3/ppo-CartPole-v1/resolve/main/replay.mp4), [原始视频2下载链接](https://huggingface.co/sb3/ppo-LunarLander-v2/resolve/main/replay.mp4)). ⬇️

In [2]:
%%html
<video autoplay controls><source src='./assets/replay1.mp4' type='video/mp4'></video>

In [3]:
%%html
<video autoplay controls><source src='./assets/replay2.mp4' type='video/mp4'></video>

💡 我们建议你使用Google Colab, 因为某些环境只适用于Ubuntu. Google Colab的免费版本很适合这个教程. 让我们开始吧! 🚀

## 这份笔记来自深度强化学习课程
![Deep Reinforcement Learning Course.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:

* 📖 研究深度强化学习的**理论和实践.**
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ **在独特的环境中训练智能体.**

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗

在深入研究笔记之前, 你需要:

🔲 📚 [阅读第8单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit8/README.md)

🔲 📚 通过阅读章节**学习Proximal Policy Optimization(PPO)** 👉 https://huggingface.co/blog/deep-rl-ppo

### 第0步: 设置GPU 💪

* 为了**更快的训练智能体, 我们将使用GPU,** 选择`修改 > 笔记本设置`
![image.png](./assets/image.png)

* `硬件加速器 > GPU`

![image.png](./assets/image1.png)

### 第1步: 安装依赖项 🔽 和 虚拟屏幕 💻

In [ ]:
!apt install ffmpeg
# 如果你使用IDE(例如PyCharm或VS Code)将不需要这些步骤.
!apt install python-opengl xvfb 
!pip install pyvirtualdisplay

In [ ]:
!pip install gym box2d-py  # 如果使用Apple M1 conda install box2d-py
!pip install huggingface_hub
!pip install imageio

In [ ]:
# 创建虚拟屏幕.
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### 第2步: 让我们使用Costa Huang的教程从头开始编写PPO
* 对于PPO的核心实现, 我们将使用优秀的[Costa Huang的教程](https://costa.sh/).
* 除此之外, 更深入的了解你可以阅读13个核心实现细节: https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/

👉 视频教程: https://youtu.be/MEt6rrxH8W4

In [6]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/MEt6rrxH8W4" ' 
     + 'title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; '
     + 'clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

* 最好的办法是先在下面的单元格中编写代码, 这样如果你的进程被关闭, 也不会丢失代码.

In [7]:
### 你的代码:

### 第3步: 添加Hugging Face集成 🤗
* 为了将我们的模型发布到Hugging Face Hub, 我们需要定义一个`package_to_hub`函数.
* 添加我们的需要将模型发布到Hugging Face Hub的依赖项.

In [ ]:
import datetime
import imageio
import json
import shutil
import tempfile
from pathlib import Path

from huggingface_hub import HfApi, upload_folder
from huggingface_hub.repocard import metadata_eval_result, metadata_save
from wasabi import Printer

msg = Printer()

* 在函数`parse_args()`中添加新参数来定义我们想要发布模型的`repo-id`.

In [ ]:
# 添加Hugging Face Hub参数.
parser.add_argument('--repo-id',
                    type=str,
                    default='ThomasSimonini/ppo-CartPole-v1',
                    help='Hugging Face Hub中模型仓库的ID{用户名/仓库名}')

* 接下来, 我们添加将模型发布到Hugging Face Hub所需的方法
* 这些方法有:
    * `_evalutate_agent()`: 评估智能体.
    * `_generate_model_card()`: 为你的智能体生成模型卡.
    * `_record_video()`: 录制智能体的回放视频.

In [ ]:
def package_to_hub(repo_id,
                   model,
                   hyperparameters,
                   eval_env,
                   video_fps=30,
                   commit_message='发布强化学习模型到Hugging Face Hub.',
                   token=None,
                   logs=None):
    """评估, 生成视频并将模型发布到Hugging Face Hub.

    此函数将执行完整的流水线:
        * 评估模型
        * 生成模型卡
        * 生成智能体的回放视频
        * 将全部内容发布到Hugging Face Hub

    Args:
        repo_id: Hugging Face Hub中模型仓库的ID
        model: 训练的模型.
        hyperparameters: 训练模型的超参数.
        eval_env: 用于评估智能体的环境.
        video_fps: 渲染回放视频的帧率.
        commit_message: 提交的信息.
        token: 发布模型的Hugging Face令牌.
        logs: 你要上传的TensorBoard日志的本地目录.
    """
    msg.info('这个函数将保存, 评估, 生成智能体回放视频, 创建模型卡并将模型发布到Hugging Face Hub.'
             '最多可能需要1分钟. 这是一项正在进行的工作, 如果你遇到BUG, 请打开一个issue.')

    # 第1步: 克隆或创建仓库.
    repo_url = HfApi().create_repo(repo_id=repo_id,
                                   token=token,
                                   private=False,
                                   exist_ok=True)

    with tempfile.TemporaryDirectory() as tmp_dir:
        tmp_dir = Path(tmp_dir)

        # 第2步: 保存模型.
        torch.save(model.state_dict(), tmp_dir / 'model.pt')

        # 第3步: 评估模型并构建JSON.
        mean_reward, std_reward = _evaluate_agent(eval_env, 10, model)

        # 首先, 获取当前时间.
        eval_datetime = datetime.datetime.now()
        eval_form_datetime = eval_datetime.isoformat()

        evaluate_data = {
            'env_id': hyperparameters.env_id,
            'mean_reward': mean_reward,
            'std_reward': std_reward,
            'n_evaluation_episodes': 10,
            'eval_datetime': eval_form_datetime
        }

        # 写入JSON文件.
        with open(tmp_dir / 'hyperparameters.json', 'w') as outfile:
            json.dump(evaluate_data, outfile)

        # 第4步: 录制回放视频.
        video_path = tmp_dir / 'replay.mp4'
        _record_video(eval_env, model, video_path, video_fps)

        # 第5步: 创建模型卡.
        generated_model_card, metadata = _generate_model_card('PPO',
                                                              hyperparameters.env_id,
                                                              mean_reward,
                                                              std_reward,
                                                              hyperparameters)
        _save_model_card(tmp_dir, generated_model_card, metadata)

        # 第6步: 如果需要则添加日志.
        if logs:
            _add_logdir(tmp_dir, Path(logs))

        msg.info(f'正在将仓库{repo_id}发布到Hugging Face Hub...')

        repo_url = upload_folder(repo_id=repo_id,
                                 folder_path=tmp_dir,
                                 path_in_repo='',
                                 commit_message=commit_message,
                                 token=token)

        msg.info(f'你的模型已经发布到Hugging Face Hub. 你可以点击链接查看的你的模型: {repo_url}')

    return repo_url


def _evaluate_agent(env, n_eval_episodes, policy):
    """用`n_eval_episodes`轮评估智能体, 并返回奖励的均值和标准差.

    Args:
        env: 评估环境.
        n_eval_episodes: 测试的总轮数.
        policy: 强化学习智能体.

    Returns:
        奖励的均值和标准差.
    """
    episode_rewards = []

    for episode in range(n_eval_episodes):
        state = env.reset()
        done = False
        total_rewards_ep = 0

        while done is False:
            state = torch.Tensor(state)
            action, _, _, _ = policy.get_action_and_value(state)
            new_state, reward, done, info = env.step(action.numpy())
            total_rewards_ep += reward

            if done:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)

    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward, std_reward


def _record_video(env, policy, out_directory, fps=30):
    images = []
    done = False
    state = env.reset()
    img = env.render(mode='rgb_array')
    images.append(img)

    while not done:
        state = torch.Tensor(state)
        # 在给定状态下, 采取具有最大期望奖励的动作(索引).
        action, _, _, _ = policy.get_action_and_value(state)
        state, reward, done, info = env.step(action)  # 我们直接使用next_state = state来记录顺序(recording logic).
        img = env.render(mode='rgb_array')
        images.append(img)

    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)


def _generate_model_card(model_name, env_id, mean_reward, std_reward, hyperparameters):
    """为Hugging Face Hub生成模型卡.

    Args:
        model_name: 模型的名称.
        env_id: 环境的名称.
        mean_reward: 奖罚的均值.
        std_reward: 奖罚的标准差.
        hyperparameters: 训练模型的超参数.
    """